In [1]:
import pandas as pd
import os
import datetime as dt

In [2]:
# Filepaths
file1 = '/Users/danvaldes/Downloads/mlb-pitch-data-20152018/atbats.csv'
file2 = '/Users/danvaldes/Downloads/mlb-pitch-data-20152018/player_names.csv'
file3 = '/Users/danvaldes/Downloads/mlb-pitch-data-20152018/pitches.csv'
file4 = '/Users/danvaldes/Downloads/mlb-pitch-data-20152018/games.csv'

# Reading CSVs to DataFrames
at_bats_df = pd.read_csv(file1)
player_names_df = pd.read_csv(file2)
pitches_df = pd.read_csv(file3)
games_df = pd.read_csv(file4)

In [12]:
# Joining first_name and last_name columns
player_names_df['name'] = player_names_df['first_name'] + ' ' + player_names_df['last_name']
player_names_final = player_names_df.drop(['first_name','last_name'], axis =1)

# Changing ID column name for merge later
player_names_final = player_names_final.rename(columns = {'id':'pitcher_id'});

# Dropping the unnecessary pitches_df columns
pitches_df = pitches_df[['start_speed','end_speed','spin_rate','break_angle','nasty','pitch_type','ab_id','b_count','s_count','pitch_num','on_1b','on_2b','on_3b']]

In [4]:
# Dropping unnecessary games_df columns
games_df = games_df[['date','g_id']]

In [5]:
# Dropping unnecessary at_bats columns
at_bats_df = at_bats_df[['ab_id','g_id','p_throws','pitcher_id','stand']]

In [6]:
# Merging games_df and at_bats_df to retrieve date
at_bats_new = pd.merge(at_bats_df, games_df, on='g_id', how='outer')

In [45]:
# Filtering at_bats_df by year, saving only at_bats from 2018 season
at_bats_new['date'] = pd.to_datetime(at_bats_new['date'])

at_bats_final = at_bats_new[at_bats_new['date'].dt.year == 2018]

at_bats_final = at_bats_final[['ab_id', 'p_throws', 'pitcher_id', 'stand']]

at_bats_final.head();

In [44]:
# Merging pitches and at_bats to create final DataFrame
pitches_final = pd.merge(pitches_df, at_bats_final, on='ab_id', how='left')

pitches_final = pitches_final.dropna(how='any')

pitches_final.head();

In [37]:
# Merge player_names with unique pitches table. Now have information on who threw each pitch
final_df = pd.merge(pitches_final, player_names_final, on='pitcher_id', how='left')

final_df['avg_speed'] = (final_df['start_speed'] + final_df['end_speed']) / 2

final_df_organized = final_df[['name','p_throws','pitch_type','avg_speed','spin_rate','break_angle','nasty','ab_id','b_count','s_count','pitch_num','on_1b','on_2b','on_3b','stand',]].\
    sort_values(by='name')

In [43]:
final_df_organized;

In [41]:
final_df_organized.to_csv('/Users/danvaldes/Desktop/pitches_final.csv', index=False, header=True)